# NISQ Experiment

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import random_unitary, Operator

from qiskit import Aer

In [3]:
#qk.IBMQ.save_account("a9ec3c697bede976217e9ff6c5c0f19a2511ea34a225bbe10f0c74bda39de845d16fa3f7c30b47c5714e2b695aded3cfd474e7c1b33056014fab89301b83d724", overwrite=True) 
#provider = qk.IBMQ.load_account()
#provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#backend = provider.get_backend("ibmq_belem")

backend = Aer.get_backend("aer_simulator")
from qiskit.providers.fake_provider import FakeManilaV2

## Ring Disk Transition, Two Qubits

In [4]:
backend = FakeManilaV2()

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []
for i in range(2):
    circuit_target = pqc_basic(n, 8).reverse_bits()
    circuit_target_list.append(circuit_target)
    
belem_2qubits_8layers = ExecuteAndCollect()
belem_2qubits_8layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

In [6]:
belem_2qubits_8layers.execute_circuits(backend=backend, shots_map = 1024, shots_spam = 1024, filename="belen_aer_test", concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
inputs_map, targets_map, inputs_spam, targets_spam, =  pickle.load(open(f"../../data/belen_aer_test1", "rb"))
    
N_map = inputs_map[0].shape[0]

d = targets_map.shape[1]
n = int(np.log2(d))

spam = SPAM(init = InitialState(d),
            povm = POVM(d),
            loss_function = ProbabilityMSE(),
            optimizer = tf.optimizers.Adam(learning_rate=0.01),
            )

spam.train(inputs = inputs_spam,
           targets = targets_spam,
           num_iter = 2000,
           verbose = False,
          )

  0%|          | 0/2000 [00:00<?, ?it/s]

7.427198454281684e-05


In [8]:
model = ModelQuantumMap(channel = KrausMap(d=d, 
                                           rank=16,
                                           spam = spam,
                                          ),
                           loss_function = ProbabilityMSE(),
                           optimizer = tf.optimizers.Adam(learning_rate=0.01),
                           logger = Logger(loss_function = ProbabilityMSE(), verbose=False),
                           )

model.train(inputs = inputs_map,
            targets = targets_map,
            num_iter = 2000,
            )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.005927726759762016 None


## Ring Disk Transition, Three Qubits

### 16 Layers

In [5]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 8).reverse_bits()
    circuit_target_list.append(circuit_target)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
belem_3qubits_16layers = ExecuteAndCollect()
belem_3qubits_16layers.setup_circuits(circuit_target_list, N_map = 2000)

belem_3qubits_16layers.execute_circuits(backend=backend, shots_map = 1024, shots_spam = 1024, filename="belem_3qubits_16layers")